
This SQL project uses SQLite and Python to answer questions about an imaginary music store, "Chinook"

albums: information about the albums that the store sells individual tracks from

tracks: information about the tracks present in an album like name, genre, composer, milliseconds etc.

artists: information about the artists (individual musicians or groups) that made the albums and tracks

customers: information about the customers of the store

employees: information about the employees of the store

genres: information about the musical genres associated with tracks

invoice Items: information about Individual tracks sold, linked to invoices

invoices: information about each sale to a customer of one or more tracks

media types: formats in which tracks are sold

playlists: information about playlists, which each consist of a set of tracks

playlist_track: relationships between playlists and tracks


1.In the invoices table, some of the cities are tied for having the most invoices. Return a sorted list of these cities and how many invoices they have.

In [ ]:
def MostInvoiceCities():
  q='''
  select BillingCity from (
  select i.BillingCity , count(*) as INVOICECOUNT from invoices i
  group by i.BillingCity
  Having count(*)=14 order by INVOICECOUNT desc )
  '''
  x=pd.read_sql_query(q,conn)
  return x['BillingCity'].tolist()

def nInvoices():
  q='''
  select INVOICECOUNT from (
  select i.BillingCity , count(*) as INVOICECOUNT from invoices i
  group by i.BillingCity order by INVOICECOUNT desc)
  '''
  ans=pd.read_sql_query(q,conn)
  return ans.iloc[0]['INVOICECOUNT']
#print(MostInvoiceCities())
print(nInvoices())


14


2. Return a list containing the first and last names of the customer who purchased the most rock tracks and the number of rock tracks they purchased.



In [ ]:
def CustomerMostRock():
  q='''
  select c.FirstName, c.LastName, i.InvoiceId , count(*) as COUNT from customers c
  join invoices i on c.CustomerId =i.CustomerId
  join invoice_items ii on i.InvoiceId=ii.InvoiceId
  join tracks t on ii.TrackId=t.TrackId
  where GenreId=1 group by i.CustomerId
  order by COUNT desc
  '''
  x=pd.read_sql_query(q,conn)
  return[ x.iloc[0]['FirstName'],x.iloc[0]['LastName'],x.iloc[0]['COUNT']]

print(CustomerMostRock())

['Eduardo', 'Martins', 29]


3. How many distinct jazz tracks were purchased?


In [ ]:
def DistinctJazzTracks():
  q='''
  select count( distinct t.TrackId) as JAZZ FROM tracks t
  join invoice_items ii on t.TrackId=ii.TrackId where GenreId=2
  '''
  x=pd.read_sql_query(q,conn)
  return x

print(DistinctJazzTracks())

   JAZZ
0    68


4. Reuturn a Python dictionary of those customers that bought more than 10 metal tracks, where the keys are their last names and the values are the number of tracks they bought.

In [ ]:
def TopMetalCustomers():
  q='''
  select c.LastName, count(*) as COUNT from customers c
  join invoices i on c.CustomerId = i.CustomerId
  join invoice_items ii on i.InvoiceId=ii.InvoiceId
  join tracks t on ii.TrackId=t.TrackId
  where t.GenreId=3
  group by c.CustomerId
  having COUNT>10
  '''
  x=pd.read_sql_query(q, conn)
  return x.set_index('LastName')['COUNT'].to_dict()

print(TopMetalCustomers())

{'Harris': 12, 'Leacock': 12, 'Schneider': 18, 'Girard': 14}


5. Write a Python function that returns the number of albums that more than 25 but less than 50 dollars worth of tracks.

In [ ]:
def CountAlbums():
    query='''
    SELECT  t.AlbumId, SUM(t.UnitPrice) AS Total
    FROM tracks t
    JOIN invoice_items ii ON t.TrackId=ii.TrackId
    GROUP BY t.AlbumId
    HAVING SUM(t.UnitPrice) > 25 AND SUM(t.UnitPrice) < 50
    '''
    ans=pd.read_sql_query(query,conn)
    return len(ans)

x=CountAlbums()
print(x)

6


6. Write a Python function that returns a sorted Python list of the genres that had sales between 50 and 100 dollars

In [ ]:
def GenresRange():
    query = '''
    SELECT g.Name, SUM(ii.UnitPrice) AS Total
    FROM genres g
    JOIN tracks t ON g.GenreId = t.GenreId
    JOIN invoice_items ii ON t.TrackId = ii.TrackId
    GROUP BY g.GenreId
    HAVING Total >= 50 AND Total <= 100 ORDER BY g.Name ASC
    '''
    ans = pd.read_sql_query(query, conn)
    return ans['Name'].tolist()

x = GenresRange()
print(x)

['Blues', 'Drama', 'Jazz', 'TV Shows']


7.Write a Python functionwhich returns a sorted list of the artist names that have exactly three albums in the database.

In [ ]:
def Artists3():
    query='''
    SELECT a.Name, COUNT(alb.AlbumId) as AlbumCount
    FROM artists a
    JOIN albums alb ON a.ArtistId=alb.ArtistId
    GROUP BY a.ArtistId
    HAVING COUNT(alb.AlbumId)=3
    ORDER BY a.Name
    '''
    ans=pd.read_sql_query(query,conn)
    return ans['Name'].tolist()
x=Artists3()
print(x)

['Audioslave', 'Berliner Philharmoniker & Herbert Von Karajan', 'Eugene Ormandy', 'Gilberto Gil', "Guns N' Roses", 'Jamiroquai', 'Miles Davis', 'Os Paralamas Do Sucesso', 'Queen', 'R.E.M.', 'Red Hot Chili Peppers', 'Santana', 'The Office', 'The Rolling Stones']


8.How many artists ahve at least one album?

In [ ]:
def CountArtistsWithAlbums():
    query='''
    SELECT COUNT(a.AlbumId), a.ArtistId
    FROM albums a
    GROUP BY a.ArtistId
    HAVING COUNT(a.AlbumId)>=1
    '''
    ans=pd.read_sql_query(query,conn)
    return len(ans)
x=CountArtistsWithAlbums()
print(x)

204


9.Write a Python which returns the name of the fifth longest album, in terms of the total milliseconds of its tracks as stored in the database.

In [ ]:
def FifthLongestAlbum():
    query = '''
    SELECT a.Title
    FROM albums a
    JOIN (
         SELECT AlbumID, SUM(Milliseconds) AS TotalLength
         FROM tracks
         GROUP BY AlbumID
    ) AS t ON a.AlbumID = t.AlbumID
    ORDER BY t.TotalLength DESC
    LIMIT 1 OFFSET 4
    '''
    answer = pd.read_sql_query(query, conn)
    return answer.iloc[0]['Title']
p1_answer=FifthLongestAlbum()
print(p1_answer)

Heroes, Season 1


10.Write a function CitiesMultipleCodes() that returns a sorted list of those billing cities (column BillingCity) in the invoices table that have more than one different BillingPostalCode.



In [ ]:
def CitiesMultipleCodes():
    query = '''
    SELECT i.BillingCity, COUNT(DISTINCT i.BillingPostalCode) AS PostalCodeCount
    FROM invoices i
    GROUP BY i.BillingCity
    HAVING COUNT(DISTINCT i.BillingPostalCode) > 1
    ORDER BY i.BillingCity ASC
    '''
    ans = pd.read_sql_query(query, conn)
    return ans['BillingCity'].tolist()
x = CitiesMultipleCodes()
print(x)

['Berlin', 'London', 'Mountain View', 'Paris', 'Prague', 'São Paulo']


11. Find the number of tracks per artist. Call this trackcount. Using a window function, partition the artists by trackcount into quintiles; that means that the 20% of artists with the most tracks are in quintile 1, etc. What artist is at the bottom of quintile 1? That is, they are in the top 20% in terms of tracks, but they have fewer tracks than any other artist in that quintile.

In [ ]:
def Quintile():
  q='''
  select * from (
  select *, ntile(5) over (order by TRACKCOUNT desc) as quintile from (
  select a.Name, count(*) as TRACKCOUNT from artists a
  join albums al on a.ArtistId=al.ArtistId
  join tracks t on al.AlbumId=t.AlbumId
  group by a.Name ) ) where quintile =1 order by TRACKCOUNT asc

  '''
  x=pd.read_sql_query(q,conn)
  return x.iloc[0]['Name']
print(Quintile())

Santana



12.Building on the previous question, when you rank artists using the rank() window function within each quintile by descending trackcount, two artists are tied for the rank of 10 in quintile 2. Who are they?

In [ ]:
def Rank():
  q='''
  select * ,rank () over (order by TRACKCOUNT desc)as rank from (
  select *, ntile(5) over (order by TRACKCOUNT desc) as quintile from (
  select a.Name, count(*) as TRACKCOUNT from artists a
  join albums al on a.ArtistId=al.ArtistId
  join tracks t on al.AlbumId=t.AlbumId
  group by a.Name ) ) where quintile=2 limit 2 offset 9

  '''
  x=pd.read_sql_query(q,conn)
  return x['Name'].tolist()
print(Rank())

['Caetano Veloso', 'Spyro Gyra']
